## Train

In [1]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

d:\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
model.gene

In [3]:
model.generate()

ValueError: `bos_token_id` has to be defined when no `input_ids` are provided.

In [3]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [4]:
input_lines = []
label_lines = []

task = 'D:/Downloads/DS_AI/VDT/MoE/Wiki'
train_file = 'train.tsv'

with open(f'{task}/{train_file}') as file:
  for line in file:
    line = line.strip().split('\t')
    input_lines.append(line[0] +'</s>')
    label_lines.append(line[1])


dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'])


Map:   0%|          | 0/13708 [00:00<?, ? examples/s]

d:\Anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [5]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=30,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )


# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37
# 
# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)


In [7]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pavt1024 (hust_edu_vn). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


KeyboardInterrupt: 

## Inference

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")


C:\Users\PAVT\AppData\Local\Temp\ipykernel_40352\466826681.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [8]:
input_lines = []
label_lines = []
with open(f'{task}/test.tsv') as file:
  for line in file:
    line = line.strip().split('\t')
    input = line[0]
    input_lines.append(input +'</s>')
    label_lines.append(line[1])



input_lines  = input_lines
label_lines = label_lines
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'])
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

Map:   0%|          | 0/3917 [00:00<?, ? examples/s]

d:\Anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/tmp/checkpoint-85675")
model.to('cuda')

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/tmp/checkpoint-85675'. Use `repo_type` argument if needed.

In [10]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()


  0%|          | 0/123 [00:00<?, ?it/s]

d:\Anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.0388713973982151, recall=0.0014577435849243276, fmeasure=0.0025457827894782973), mid=Score(precision=0.044307957325496805, recall=0.0018562674367500302, fmeasure=0.003117168279416901), high=Score(precision=0.04979067929739864, recall=0.0022846298077193222, fmeasure=0.003761243082587142)),
 'rouge2': AggregateScore(low=Score(precision=0.007878147231865345, recall=0.00028794975525877947, fmeasure=0.0005189681875965626), mid=Score(precision=0.00994247965019667, recall=0.0003573387077727865, fmeasure=0.0006386691865039958), high=Score(precision=0.012326584637928692, recall=0.0004302498977247224, fmeasure=0.0007617540528396785)),
 'rougeL': AggregateScore(low=Score(precision=0.03782646216623352, recall=0.001323275560592332, fmeasure=0.0023099594835722095), mid=Score(precision=0.042837660211217814, recall=0.0016471926561966075, fmeasure=0.0027981403856112154), high=Score(precision=0.048372634360663545, recall=0.0020091981207909053, fmeasure=0.0

In [11]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]


[{'rouge1': 0.003117168279416901},
 {'rouge2': 0.0006386691865039958},
 {'rougeL': 0.0027981403856112154},
 {'rougeLsum': 0.002798132373285429}]